## Final Project for the Course Applied Data Science Capstone by IBM
### Student: Fábio Vieweger Vasques

__Description__: This project has as a objective show how countries other than those with historical tradition on wine production are established around the globe.

Wine production is divided into New World and Old World. Old World is formed by countries with a long-established history of wine production, like France, Italy, Germany, Spain and Portugal. New World wines are those produced outside this traditional winegrowing area.
So, for this study, we will check the size the New World is getting, looking for vineyards around Argentina, Australia, Brasil, Canada, Chile, Mexico, New Zealand, South Africa and the United States. We will take a closer look how it is distributed aroud the globe and we will see what those vineyards has as similarities.
To be abble to accomplish this objective, we will use Foursquare API's a few times, in a loop, to locate Vineyards within those New World producers.

## About the strategie used for this notebook:
Foursquare have a great sample of vineyards around the globe available on their API. So I will parse a few times those APIs, gethering as much information possible.
As there is a limit of daily access of foursquare API, a DB2 database has been created to store the data retrieved from Foursquare. Within this database data has been cleaned up, removing duplicities and some standards have been applied to rows which were lacking some information. All vineyards retrivied have latitude and longitude information to be plotted on a map.

## Target for this Study
This notebook has the intention to provide information for wine enthusiast, bringing information that is not available gathered in other places.

In [ ]:
### Libraries and Dependecies Needed
!pip install folium
import pandas as pd # store dataframes
import requests # retrieve information from API
from pandas.io.json import json_normalize # Threat the data retrieved from the API
import ibm_db # Library to work with DB2 database
import ibm_db_dbi # Library to work DB2 with Pandas
import folium # Map library
from folium import plugins
import matplotlib as mpl
%matplotlib inline
print('All imports are complete')

In [ ]:
# Create database connection to persist the API data on db2 cloud
dsn_hostname = "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "ljb20609"        # e.g. "abc12345"
dsn_pwd = "nst1kws8v63^1422"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000"
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
print('Connection variables are set')

#Configure DSN
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

In [ ]:
#Check connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [ ]:
### Create the table where data from the API will be stored
createTable = ('create table vineyards (id_forsquare varchar(50),'
                                        'vineyard_address varchar(200),'
                                        'abreviation_country varchar(5),'
                                        'city varchar(100),'
                                        'country varchar(100),'
                                        'latitude decimal(8,6),'
                                        'longitude decimal(8,6),'
                                        'city_state varchar (100),'
                                        'Vineyard_name varchar (200))')
createStmt = ibm_db.exec_immediate(conn, createTable)
createStmt
ibm_db.close

In [ ]:
# SET API parameters
client_id = 'R33E0XLKZ3PP5RI4OOQW5WDDISIYGOFCB5FBWVOMGCSDURKO'
client_secret = 'F4C0LBGKN2X4D5YETF22AREE1EM2PRGGYV222J02A13PSJF3'
categoryId='4bf58dd8d48988d1de941735' #Category for vineyards
version='20200203'

print ('ID for client is:', client_id, '\nClient secret is: ', client_secret)

In [ ]:
# New World Wine Countries: Argentina, Australia, Brasil, Canada, Chile, Mexico, New Zealand, South Africa and the United States.
new_world = ('Argentina', 'Australia', 'Brasil', 'Canada', 'Chile', 'Mexico', 'New Zealand', 'South Africa', 'United States')

In [ ]:
df_vineyards = pd.DataFrame()
for country in new_world:
    #set the API parameters to start the iteration
    offset = 0
    near = country
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&categoryId={}&offset={}'.format(client_id, client_secret, near, version, categoryId, offset)
    results = requests.get(url).json()
    while 'warning' not in results['response']:
        venues = results['response']['groups'][0]['items']
        df_vineyards = json_normalize(venues)
        insertVinho = ('insert into vineyards values (?, ?, ?, ?, ?, ?, ?, ?, ?)')
        prep_stmt = ibm_db.prepare(conn, insertVinho)
        for i in df_vineyards.index:
            ibm_db.bind_param(prep_stmt, 1, df_vineyards['venue.id'][i])
            ibm_db.bind_param(prep_stmt, 2, df_vineyards['venue.location.address'][i])
            ibm_db.bind_param(prep_stmt, 3, df_vineyards['venue.location.cc'][i])
            ibm_db.bind_param(prep_stmt, 4, df_vineyards['venue.location.city'][i])
            ibm_db.bind_param(prep_stmt, 5, df_vineyards['venue.location.country'][i])
            ibm_db.bind_param(prep_stmt, 6, df_vineyards['venue.location.lat'][i])
            ibm_db.bind_param(prep_stmt, 7, df_vineyards['venue.location.lng'][i])
            ibm_db.bind_param(prep_stmt, 8, df_vineyards['venue.location.state'][i])
            ibm_db.bind_param(prep_stmt, 9, df_vineyards['venue.name'][i])
            ibm_db.execute(prep_stmt)
        ibm_db.close
        offset = offset+1
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={}&v={}&categoryId={}&offset={}'.format(client_id, client_secret, near, version, categoryId, offset)
        results = requests.get(url).json()

In [ ]:
# Create some DB2 connection
pconn = ibm_db_dbi.Connection(conn)
query = 'select distinct * from vineyards'

In [ ]:
df_new_world = pd.read_sql(query, pconn)

In [ ]:
updateAddress = "update vineyards set vineyard_address = 'Not defined' where vineyard_address = '1,#QNAN000000000E+0'"
script = ibm_db.prepare(conn, updateAddress)
ibm_db.execute(script)

In [ ]:
updateCity = "update vineyards set city = 'Not defined' where city = '1,#QNAN000000000E+0'"
script = ibm_db.prepare(conn, updateCity)
ibm_db.execute(script)

In [ ]:
updateState = "update vineyards set city_state = 'Not defined' where city_state = '1,#QNAN000000000E+0'"
script = ibm_db.prepare(conn, updateState)
ibm_db.execute(script)

In [ ]:
#Now let's create a table named NEW_WORLD with distinct data from table VINEYARDS. This is the table we will be using, once database processing with distinct will need much more processing than without it.
createNewWolrd = 'create table NEW_WORLD as (select distinct * from vineyards) with data'
script = ibm_db.prepare(conn, createNewWolrd)
ibm_db.execute(script)

Now let's take a look how our information are set on the dataframe, result from the New World vineyards.

In [ ]:
query = 'select * from NEW_WORLD'
df_new_world = pd.read_sql(query, pconn)
df_new_world.head()

Our sample, as you can see on the list below, if formed by 9 significant countries of the New World wine production

### New World Countries

In [68]:
count_new_world = df_new_world['COUNTRY'].value_counts()
df_count_new_world = pd.DataFrame(count_new_world)
df_count_new_world.rename(columns = {'COUNTRY':'Quantity'}, inplace=True)
df_count_new_world

,Quantity
United States,303
Australia,136
iNingizimu Afrika,110
Canada,84
Chile,73
New Zealand,72
México,68
Argentina,45
Brasil,41


So as seen above, we have loaded 303 vineyards from United States, and we can check how it is distributed on the map.

In [ ]:
df_united_states = df_new_world.loc[df_new_world['COUNTRY']=='United States']

In [69]:
us_map = folium.Map(location=(37.0902,-105.7129),
                      zoom_start = 4,
                      width='100%',
                      height='100%')
us_wines = plugins.MarkerCluster().add_to(us_map)
labels = list(df_united_states.VINEYARD_NAME)
latitudes = list(df_united_states.LATITUDE)
longitudes = list(df_united_states.LONGITUDE)


for lat, lng, name, in zip(latitudes, longitudes, labels):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=name,
    ).add_to(us_wines)

us_map

Well, is no surprise to see that more than 50% of vineyards loaded from Foursquare API are located at California, but definitely I wasn't expecting to see so much states which produces wines around US. As you can see at the map above, more than 60 vineyards founded are located on Virginia and New York state.  
Now, after this first look at the US map and its vineyards, let's take a look how it looks like around the globe.

In [73]:
old_world_query = 'select * from old_world'
df_old_world = pd.read_sql(old_world_query, pconn)

In [72]:
wine_map = folium.Map(location=(0,0),
                      zoom_start = 2,
                      width='100%',
                      height='100%')
incidents = plugins.MarkerCluster().add_to(wine_map)
labels_new = list(df_new_world.VINEYARD_NAME)
latitudes_new = list(df_new_world.LATITUDE)
longitudes_new = list(df_new_world.LONGITUDE)

labels_old = list(df_old_world.VINEYARD_NAME)
latitudes_old = list(df_old_world.LATITUDE)
longitudes_old = list(df_old_world.LONGITUDE)


for lat, lng, name, in zip(latitudes_new, longitudes_new, labels_new):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=name,
    ).add_to(incidents)


for lat, lng, name, in zip(latitudes_old, longitudes_old, labels_old):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=name,
    ).add_to(incidents)
    
wine_map

We can notice looking the map above, that almost all the vineyards are concentrated at the same latitude lines.  
And looking at the map below, and looking the mean temperature of each place 

<img src = "https://external-preview.redd.it/iiwYEl1aPfhd6fsGm2qMuj1hGvIogmt9L74mNcKX8Pw.png?auto=webp&s=2d4f0c071e6b224f5ce07045d9c3e7f5e42b22cf">